In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
from pathlib import Path
import fiona 
import numpy as np
import pandas as pd
import logging
# Plotting
import matplotlib.pyplot as plt
# Pour la basemap
import contextily as cx
# Potentiel solaire package

In [ ]:

from potentiel_solaire.attach_buildings_to_schools import attach_buildings_to_schools
from potentiel_solaire.constants import DATA_FOLDER
from potentiel_solaire.constants import CRS, ALGORITHME_FOLDER, DATA_FOLDER
from potentiel_solaire.sources.bd_topo import extract_bd_topo, get_topo_zones_of_interest, \
    get_topo_buildings_of_interest
from potentiel_solaire.sources.bd_pci import extract_bd_pci
from potentiel_solaire.sources.bd_solar_irradiation import extract_bd_irradiation
from potentiel_solaire.sources.schools_establishments import extract_schools_establishments, \
    get_schools_establishments_of_interest
from potentiel_solaire.features.solar_potential import calculate_solar_potential
from potentiel_solaire.aggregate import aggregate_solar_potential_by
from potentiel_solaire.logger import get_logger

logger = get_logger() 
logger.setLevel(logging.ERROR)

In [ ]:
code_departement = "093"
compare_with_idf = True

# Imports & setup

In [ ]:
dpts = gpd.read_file(DATA_FOLDER / "contour-des-departements.geojson").to_crs(4326)
ShapeNT = dpts[dpts.code == "93"].geometry
fig, ax = plt.subplots()
dpts[dpts.code == "93"].plot(column="nom",legend=True, categorical = True, ax = ax, color="green",alpha=0.5)
cx.add_basemap(ax, crs=dpts.crs)
fig.show()

In [ ]:
# get data and get paths
schools_establishments_path = extract_schools_establishments()
bd_topo_path = extract_bd_topo(code_departement=code_departement)
bd_pci_path = extract_bd_pci(code_departement=code_departement)

In [ ]:
schools_establishments = get_schools_establishments_of_interest(
    schools_filepath=schools_establishments_path,
    code_departement=code_departement,
    types_etablissements=['Ecole', 'Lycée', 'Collège'],
    statut_public_prive="Public",
    etat="OUVERT",
    crs=CRS
)
nb_schools = schools_establishments.shape[0]
print(f"Nb d'établissements scolaires: {nb_schools}")


In [ ]:
educational_zones = get_topo_zones_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=ShapeNT,
    categories=["Science et enseignement"],
    natures=['Collège', 'Lycée', 'Enseignement primaire'],
    crs=CRS
)
nb_educational_zones = educational_zones.shape[0]
print("Nb de zones d'éducations: ", nb_educational_zones)

In [ ]:
# Preparation 'educational_zones'
educational_zones = educational_zones.dropna(subset=["identifiants_sources"])
educational_zones["identifiants_sources"] = educational_zones["identifiants_sources"].apply(lambda x: [y.strip() for y in str(x).replace("MEN:","").split("/")])
educational_zones = educational_zones.explode('identifiants_sources')

# Recollement
educational_zones_attached_to_schools = pd.merge(
    educational_zones,
    schools_establishments[["identifiant_de_l_etablissement","nom_etablissement"]],
    left_on='identifiants_sources', right_on='identifiant_de_l_etablissement',
    how="inner"
)

educational_zones_attached_to_schools = educational_zones_attached_to_schools.dropna(subset=["cleabs","identifiants_sources"])

In [ ]:
schools_establishments = schools_establishments.dropna(subset=["identifiant_de_l_etablissement"])
schools_establishments